## Deploying a CPLEX export file with MIP starts with Watson Machine Learning

This notebook shows you how to deploy a CPLEX export file with MIP starts, create and monitor jobs, and get solution and logs using the Watson Machine Learning Python Client.

This notebook runs on Python.

## Table of Contents
1. [Install the Watson Machine Learning client API](#setup)
2. [Create a client instance](#create)
3. [Prepare your model archive](#prepare)
4. [Upload your model on Watson Machine Learning](#upload)
5. [Create a deployment](#deploy)
6. [Create and monitor a job with inline data for your deployed model](#job)
7. [Summary](#summary)

<a id='setup'></a>
### Set up the Watson Machine Learning client

Before you use the sample code in this notebook, you must:

- create a <a href="https://cloud.ibm.com/catalog/services/machine-learning" target="_blank" rel="noopener noreferrer">Watson Machine Learning (WML) Service</a> instance. A free plan is offered and information about how to create the instance can be found at <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-setup.html" target="_blank" rel="noopener noreferrer"> https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/wml-setup.html.</a>


Install and then import the Watson Machine Learning client library.

In [ ]:
# Install WML client API

!pip install ibm-watson-machine-learning

In [ ]:
from ibm_watson_machine_learning import APIClient

<a id='create'></a>
### Create a client instance

Use your Watson Machine Learning credentials. You can find information on how to get your API key <a href="https://dataplatform.cloud.ibm.com/docs/content/DO/WML_Deployment/DeployModelRest.html?audience=wdp">here</a> in the second step of the "Before you begin" section and instance's URL <a href="https://dataplatform.cloud.ibm.com/docs/content/DO/WML_Deployment/DeployModelRest.html?audience=wdp">here</a> in the third step of the "Procedure" section.

In [ ]:
# Instantiate a client using credentials
wml_credentials = {
      "apikey": "<API_key>",
      "url": "<instance_url>"
}

client = APIClient(wml_credentials)

In [ ]:
client.version

<a id='prepare'></a>
### Prepare your model archive

Use the `write_file` command to write these models to a `burger.lp` and `burger.mst` file. 

Use the `tar` command to create a tar archive.

In [ ]:
%mkdir model

In [ ]:
%%writefile burger.lp

\Problem name: good_burger

Maximize
 obj: 0.250000000000 how_many_beef + 0.150000000000 how_many_bun
      + 0.100000000000 how_many_cheese + 0.090000000000 how_many_onions
      + 0.030000000000 how_many_pickles + 0.040000000000 how_many_lettuce
      + 0.020000000000 how_many_ketchup + 0.040000000000 how_many_tomato
Subject To
 c1: how_many_beef <= 5
 c2: how_many_beef >= 1
 c3: how_many_bun <= 5
 c4: how_many_bun >= 1
 c5: how_many_cheese <= 5
 c6: how_many_cheese >= 1
 c7: how_many_onions <= 5
 c8: how_many_onions >= 1
 c9: how_many_pickles <= 5
 c10: how_many_pickles >= 1
 c11: how_many_lettuce <= 5
 c12: how_many_lettuce >= 1
 c13: how_many_ketchup <= 5
 c14: how_many_ketchup >= 1
 c15: how_many_tomato <= 5
 c16: how_many_tomato >= 1
 c17: 50 how_many_beef + 330 how_many_bun + 310 how_many_cheese
      + how_many_onions + 260 how_many_pickles + 3 how_many_lettuce
      + 160 how_many_ketchup + 3 how_many_tomato <= 2999
 c18: 17 how_many_beef + 9 how_many_bun + 6 how_many_cheese + 2 how_many_onions
       <= 149
 c19: 220 how_many_beef + 260 how_many_bun + 70 how_many_cheese
      + 10 how_many_onions + 5 how_many_pickles + 4 how_many_lettuce
      + 20 how_many_ketchup + 9 how_many_tomato <= 2999
 c20: how_many_ketchup - how_many_lettuce = 0
 c21: how_many_pickles - how_many_tomato = 0

Bounds

Generals
 how_many_beef how_many_bun how_many_cheese how_many_onions how_many_pickles
 how_many_lettuce how_many_ketchup how_many_tomato
End

Partial MIP starts for this model.

In [ ]:
%%writefile burger.mst

<CPLEXSolution version="1.0">
 <header problemName="good_burger"/>
 <variables>
  <variable name="how_many_beef" index="0" value="5"/>
  <variable name="how_many_bun" index="1" value="5"/>
  <variable name="how_many_cheese" index="2" value="1"/>
  <variable name="how_many_onions" index="3" value="5"/>
 </variables>
</CPLEXSolution>

In [ ]:

import tarfile
def reset(tarinfo):
    tarinfo.uid = tarinfo.gid = 0
    tarinfo.uname = tarinfo.gname = "root"
    return tarinfo
tar = tarfile.open("model.tar.gz", "w:gz")
tar.add("burger.lp", arcname="burger.lp", filter=reset)
tar.add("burger.mst", arcname="burger.mst", filter=reset)
tar.close()


<a id='upload'></a>
### Upload your model on Watson Machine Learning

Store model in Watson Machine Learning with:
* the tar archive previously created,
* metadata including the model type and runtime

Get the `model_uid`.

In [ ]:
# Find the space ID

space_name = "<space_name>"

space_id = [x['metadata']['id'] for x in client.spaces.get_details()['resources'] if x['entity']['name'] == space_name][0]

client.set.default_space(space_id)

In [ ]:
mnist_metadata = {
    client.repository.ModelMetaNames.NAME: "BurgerProduction",
    client.repository.ModelMetaNames.DESCRIPTION: "Model for Burger Production",
    client.repository.ModelMetaNames.TYPE: "do-cplex_22.1",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: client.software_specifications.get_uid_by_name("do_22.1"),
}

model_details = client.repository.store_model(model='/home/wsuser/work/model.tar.gz', meta_props=mnist_metadata)
#model='/home/wsuser/work/model.tar.gz', 
model_uid = client.repository.get_model_id(model_details)

<a id='deploy'></a>
### Create a deployment 

Create a batch deployment for the model, providing information such as:
* the maximum number of compute nodes
* the T-shirt size of the compute nodes

Get the `deployment_uid`.

In [ ]:
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "BurgerProduction Deployment",
    client.deployments.ConfigurationMetaNames.DESCRIPTION: "BurgerProduction Deployment",
    client.deployments.ConfigurationMetaNames.BATCH: {},
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {'name': 'S', 'num_nodes': 1}
}

deployment_details = client.deployments.create(model_uid, meta_props=meta_props)

deployment_uid = client.deployments.get_uid(deployment_details)

# print deployment id if needed
# print( deployment_uid )

In [ ]:
# List all existing deployments

client.deployments.list()

<a id='job'></a>
### Create and monitor a job with inline data for your deployed model

Create a payload containing inline input data.

Create a new job with this payload and the deployment.

Get the `job_uid`.

In [ ]:
solve_payload = {
    "solve_parameters" : {
                "oaas.logAttachmentName":"log.txt",
                "oaas.logTailEnabled":"true",
                "oaas.resultsFormat": "XML"
            },
    client.deployments.DecisionOptimizationMetaNames.INPUT_DATA: [
    ],
    client.deployments.DecisionOptimizationMetaNames.OUTPUT_DATA: [
    {
        "id":".*\.xml"
    },
    {
        "id":"log.txt"
    }
    ]
}
job_details = client.deployments.create_job(deployment_uid, solve_payload)
job_uid = client.deployments.get_job_uid(job_details)

Display job status until it is completed.

The first job of a new deployment might take some time as a compute node must be started.

In [ ]:
from time import sleep

while job_details['entity']['decision_optimization']['status']['state'] not in ['completed', 'failed', 'canceled']:
    print(job_details['entity']['decision_optimization']['status']['state'] + '...')
    sleep(5)
    job_details=client.deployments.get_job_details(job_uid)

print(job_details['entity']['decision_optimization']['solve_state']['solve_status'])

#### Check the MIP Starts were used by the engine

In [ ]:
import base64
import io

output_data = job_details['entity']['decision_optimization']['output_data']

starts_in_logs = [line for o in output_data if o['id'] == 'log.txt' for line in io.BytesIO(base64.b64decode(o['content'])) if "start" in str(line) ]
if len(starts_in_logs) == 0:
    print("Something went wrong")
elif len(starts_in_logs) == 1:
    print("MIP starts were provided to the job but engine rejected them")
    print("Something went wrong")
elif len(starts_in_logs) == 2:
    print("MIP starts were provided to the job and were used by the engine")
    for d in starts_in_logs:
        print(d)
else:
    print("Something went wrong")

### Delete the deployment

Use the following method to delete the deployment.

In [ ]:
client.deployments.delete(deployment_uid)

<a id='summary'></a>
### Summary and next steps

You've successfully completed this notebook! 

You've learned how to:

- work with the Watson Machine Learning client
- prepare your model archive and upload your model on Watson Machine Learning
- create a deployment
- create and monitor a job with inline data for your deployed model

Check out our online documentation at <a href="https://dataplatform.cloud.ibm.com/docs" target="_blank" rel="noopener noreferrer">https://dataplatform.cloud.ibm.com/docs</a> for more samples, tutorials and documentation. 

**Note**
In this sample, the optimization content (LP file + MIP Starts file) was uploaded as Watson Machine Learning model content (call to store_model). When the job is triggered, this model content is used: there is no need for any additional input data.

Another possible implementation could be to create an empty Watson Machine Learning model (call store_model with an empty tar.gz file), then when triggering the job, pass the optimization content (LP file + MIP Starts file) as the job input data (client.deployments.DecisionOptimizationMetaNames.INPUT_DATA).

<hr>
Copyright © 2019, 2023. This notebook and its source code are released under the terms of the MIT License.